In [48]:
import os
import pandas as pd
import numpy as np
import re
import json
#from pandas.io.json import json_normalize
from collections import Counter # for counting words

In [12]:
# navigate to the folder that holds information of the files
dir_t = '/Users/yizhao/Documents/spring_board/best_seller'
os.chdir(dir_t)
f = []
for (dirpath, dirnames, filenames) in os.walk(os.getcwd()):
    f.extend(filenames)
    break

In [13]:
print('\n'.join(f))

A_Dog's_Purpose:_A_Novel_for_Humans.json
A_Gentleman_in_Moscow:_A_Novel.json
A_Man_Called_Ove:_A_Novel.json
Big_Little_Lies.json
Bullseye_(Michael_Bennett).json
Echoes_in_Death:_An_Eve_Dallas_Novel_(In_Death,_Book_44).json
Fifty_Shades_Darker.json
Gunmetal_Gray_(Gray_Man).json
Heartbreak_Hotel:_An_Alex_Delaware_Novel.json
Hillbilly_Elegy:_A_Memoir_of_a_Family_and_Culture_in_Crisis.json
Killing_the_Rising_Sun:_How_America_Vanquished_World_War_II_Japan.json
Lincoln_in_the_Bardo:_A_Novel.json
Milk_and_Honey.json
My_Grandmother_Asked_Me_to_Tell_You_She's_Sorry.json
My_Not_So_Perfect_Life:_A_Novel.json
Never_Never.json
Norse_Mythology.json
Right_Behind_You.json
Small_Great_Things:_A_Novel.json
Sting.json
The_Girl_Before:_A_Novel.json
The_Handmaid's_Tale.json
The_Mistress:_A_Novel.json
The_Shack:_Where_Tragedy_Confronts_Eternity.json
The_Underground_Railroad_(National_Book_Award_Winner)_(Oprah's_Book_Club):_A_Novel.json
The_Whistler.json
This_Life_I_Live:_One_Man's_Extraordinary,_Ordinary_Li

In [11]:
json_files = list(filter(lambda k: '.json' in k, f)) # make sure to contain only json files
print(len(json_files))

28


In [15]:
# go through the list of json files to load them into a long list of dictionaries

temp_json = [json.load((open(x))) for x in json_files]

In [23]:
temp_json[0].keys()

dict_keys(['publish_date', 'reviews', 'average_rating', 'sales_rank', 'publisher', 'name', 'review_titles', 'rating_perc', 'url', 'number_reviews', 'ASIN'])

In [24]:
temp_json[0].get('rating_perc')

{'1 star': '2%',
 '2 star': '1%',
 '3 star': '3%',
 '4 star': '8%',
 '5 star': '86%'}

In [30]:
best_seller_dat = pd.DataFrame(temp_json)
best_seller_dat.head() 


,ASIN,average_rating,name,number_reviews,publish_date,publisher,rating_perc,review_titles,reviews,sales_rank,url
0,0765330342,4.6,A Dog's Purpose: A Novel for Humans,3,"[May 24, 2011]",Forge Books,"{'4 star': '8%', '3 star': '3%', '5 star': '86...","[A Must Read!, THE BEST BOOK EVER!!!!!!!!!!, A...","[Ummmm, I almost cannot fathom how the depths ...",144,https://www.amazon.com/dp/product/0765330342
1,0670026190,4.7,A Gentleman in Moscow: A Novel,1,"[September 6, 2016]",Viking,"{'4 star': '13%', '3 star': '3%', '5 star': '8...","[The best book of 2016, A missed opportunity t...","[5++ stars, A Gentleman in Moscow will stay wi...",78,https://www.amazon.com/dp/product/0670026190
2,1476738025,4.6,A Man Called Ove: A Novel,10,"[May 5, 2015]",Washington Square Press,"{'4 star': '17%', '3 star': '4%', '5 star': '7...","[A Book for everyone, Pretend Ove is 79, Moved...",[There is a part of me that didn't think I nee...,11,https://www.amazon.com/dp/product/1476738025
3,0425274861,4.5,Big Little Lies,13,"[August 11, 2015]",Berkley,"{'4 star': '26%', '3 star': '6%', '5 star': '6...",[Another amazing read by Liane Moriarty..Defin...,[I absolutely loved this book! 's. Liane Moria...,18,https://www.amazon.com/dp/product/0425274861
4,1455585300,4.3,Bullseye (Michael Bennett),622,"[February 14, 2017]",Grand Central Publishing,"{'4 star': '20%', '3 star': '11%', '5 star': '...","[Disappointing, Worst of the Michael Bennett b...",[I have read all of the books in the Michael B...,1,https://www.amazon.com/dp/product/1455585300


#### This is looking at all books from one book list, namely the "best sellers" list. There are other lists such as biographies, lifetime must reads, etc. All those json files could be stored in one single directory and loaded here.
#### The corresponding book list information of each book could be retrieved from the meta data table. Once the meta data table gets joined into this table of reviews info, the book list information will be here as well. 

In [40]:
# organize the rating breakups

rating_breakup = best_seller_dat['rating_perc'].apply(pd.Series) # spread into columns
rating_breakup = rating_breakup.fillna('0%') # fill NA values with zero
rating_breakup = rating_breakup.replace('%','',regex=True).astype('float')/100 # convert into decimals
rating_breakup.head()

,1 star,2 star,3 star,4 star,5 star
0,0.02,0.01,0.03,0.08,0.86
1,0.02,0.01,0.03,0.13,0.81
2,0.02,0.02,0.04,0.17,0.75
3,0.01,0.02,0.06,0.26,0.65
4,0.04,0.05,0.11,0.20,0.60


In [59]:
# parse out the bag of words for both Titles and the Reviews
# start with one book
test_list = best_seller_dat.review_titles[0]
one_string = '.'.join(test_list)
print(one_string[:50])

A Must Read!.THE BEST BOOK EVER!!!!!!!!!!.An absol


In [50]:
# keep only the words  

clean_string = re.sub(r'\W|\d',' ', one_string).lower() # and change to all lower cases
clean_string[:100]

'a must read  the best book ever           an absolute must book for anyone that has lost a pet a dog'

In [54]:
# break and then break them into a list of words
words = re.split(r'\s+', clean_string)

# remove words that are less than 3 letters long

filtered = [ v for v in words if len(v)>2 ]
filtered[:5]

['must', 'read', 'the', 'best', 'book']

In [57]:
# get a dictionary of word occurrences
dict(Counter(filtered))

{'finished': 1,
 'tear': 15,
 'stuck': 1,
 'know': 8,
 'delight': 7,
 'before': 7,
 'hug': 5,
 'feast': 1,
 'non': 1,
 'chapter': 2,
 'got': 4,
 'fulfilling': 1,
 'pets': 3,
 'won': 3,
 'four': 48,
 'purpose': 240,
 'comes': 1,
 'having': 2,
 'refreshing': 2,
 'for': 246,
 'haven': 2,
 'owned': 1,
 'calling': 1,
 'used': 1,
 'grader': 1,
 'rule': 1,
 'copies': 4,
 'biggest': 1,
 'expressions': 1,
 'price': 2,
 'laughter': 4,
 'shelters': 1,
 'exhilarating': 1,
 'pulls': 1,
 'putting': 2,
 'disappointing': 1,
 'whether': 3,
 'teachers': 1,
 'collection': 1,
 'pretty': 4,
 'devoured': 2,
 'good': 137,
 'tearful': 1,
 'many': 9,
 'inspired': 1,
 'loses': 1,
 'justice': 1,
 'captivating': 4,
 'turner': 4,
 'alert': 2,
 'powerful': 3,
 'maybe': 2,
 'reads': 3,
 'doubtors': 1,
 'dance': 1,
 'taking': 1,
 'timing': 1,
 'unto': 1,
 'time': 37,
 'kudos': 1,
 'extra': 1,
 'return': 1,
 'should': 13,
 'trip': 1,
 'dog': 615,
 'starting': 1,
 'unusual': 4,
 'lover': 81,
 'thru': 2,
 'beautiful': 3

In [60]:
# now define a function that intakes a list of strings and outputs a dictionary of word frequencies
def word_count(test_list):
    one_string = '.'.join(test_list)
    clean_string = re.sub(r'\W|\d',' ', one_string).lower() # and change to all lower cases
    # break and then break them into a list of words
    words = re.split(r'\s+', clean_string)

    # remove words that are less than 3 letters long
    filtered = [ v for v in words if len(v)>2 ]
    # get a dictionary of word occurrences
    return dict(Counter(filtered))



In [61]:
# apply the word count function to all the books and get a list of dictionaries
titles_word_c = [word_count(x) for x in best_seller_dat.review_titles]

In [62]:
# Unify all books' word counts so that they have the same length of keys
# 1. get the union of all keys, 
# 2. add the missing keys in individual dictionaries

def unify_keys(L): # L is a list of dictionaries
    allkeys = frozenset().union(*L)
    for i in L:
        for j in allkeys:
            if j not in i:
                i[j]=0
    return L

In [65]:
# try verify the list of words 
titles_word_unified = unify_keys(titles_word_c)
len(titles_word_unified[0].keys()) # here is the number of all present words among the books

8580

In [67]:
spread_titles = pd.Series(titles_word_unified).apply(pd.Series) # spread into columns with each word as a column header
spread_titles.head()

,aaa,aaah,aanaother,abandon,abbey,abd,abduction,abe,abiding,ability,...,zakaria,zero,zesty,zing,zinger,zjapan,zone,zweig,zzzzzzzz,ŕecomend
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0,0,0,2,...,0,0,0,0,1,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
spread_titles.shape # 28 rows by 8580 columns

(28, 8580)

In [73]:
spread_titles.sum().sort_values(ascending=False)[:20]
# once we have multiple book lists, 
#the words could be grouped by the book list first before summing their occurrences

stars    16749
five     12561
read      8247
book      8237
the       7454
great     5224
and       4987
this      4322
good      3242
four      2893
story     2454
loved     2184
love      1887
for       1809
one       1755
not       1743
was       1676
very      1637
you       1576
but       1576
dtype: int64